In [33]:
import requests
import pandas as pd
import time

def fetch_books(query, max_total=1000, delay=0.2):
    all_books = []
    max_results = 40

    for start_index in range(0, max_total, max_results):
        params = {
            'q': query,
            'startIndex': start_index,
            'maxResults': max_results,
        }
        response = requests.get('https://www.googleapis.com/books/v1/volumes', params=params)

        if response.status_code != 200:
            print(f"Ошибка запроса: {response.status_code}")
            break

        data = response.json()

        # Показываем общее количество найденных книг при первом запросе
        if start_index == 0:
            total = data.get('totalItems', '?')
            print(f"🔎 Всего найдено книг по запросу '{query}': {total}")

        items = data.get('items', [])
        print(f"📥 startIndex={start_index}, получено книг: {len(items)}")

        if not items:
            print("🚫 Больше результатов нет, останавливаем загрузку.")
            break

        for item in items:
            info = item.get('volumeInfo', {})
            all_books.append({
                'title': info.get('title'),
                'authors': ', '.join(info.get('authors', [])),
                'publishedDate': info.get('publishedDate'),
                'categories': ', '.join(info.get('categories', [])) if 'categories' in info else None,
                'pageCount': info.get('pageCount'),
                'averageRating': info.get('averageRating'),
                'ratingsCount': info.get('ratingsCount'),
                'description': info.get('description')
            })

        time.sleep(delay)

    print(f"✅ Загружено всего: {len(all_books)} книг\n")
    return pd.DataFrame(all_books)

# 🔍 Запрос
query = "detective"
df = fetch_books(query, max_total=1000)

# 💾 Сохраняем
filename = f"books_{query.replace(' ', '_')}.csv"
df.to_csv(filename, index=False, encoding='utf-8')
print(f"💾 Сохранено {len(df)} записей в файл '{filename}'")


🔎 Всего найдено книг по запросу 'detective': 1000000
📥 startIndex=0, получено книг: 40
📥 startIndex=40, получено книг: 40
📥 startIndex=80, получено книг: 40
📥 startIndex=120, получено книг: 40
📥 startIndex=160, получено книг: 40
📥 startIndex=200, получено книг: 40
📥 startIndex=240, получено книг: 40
📥 startIndex=280, получено книг: 40
📥 startIndex=320, получено книг: 0
🚫 Больше результатов нет, останавливаем загрузку.
✅ Загружено всего: 320 книг

💾 Сохранено 320 записей в файл 'books_detective.csv'


In [31]:
df.head()

,title,authors,publishedDate,categories,pageCount,averageRating,ratingsCount,description
0,Clown Horror (Russian Edition),I. D. Oro,None,Young Adult Fiction,433.0,NaN,NaN,президент Führer Devil-Eggs присоединяется к Л...
1,Clown Horror 152 (Russian Edition),I. D. Oro,2022-01-02,Young Adult Fiction,132.0,NaN,NaN,"Osvaldo гений, даже если его жена Sariyah люби..."
2,Clown Horror 150 (Russian Edition),I. D. Oro,2022-01-02,Young Adult Fiction,133.0,NaN,NaN,В параллельной вселенной партия «Правительство...
3,Horror Fiction,Gina Wisker,2005-07-13,Literary Criticism,308.0,NaN,NaN,This is a series of introductory books about d...
4,Presenting Young Adult Horror Fiction,Cosette N. Kies,1992,American fiction,232.0,NaN,NaN,"Horror, an increasingly popular subject for bo..."


In [34]:
all_queries = [
    "detective AND author:Agatha",
    "detective AND author:Doyle",
    "detective AND thriller",
    "detective AND mystery"
]

all_dfs = []

for q in all_queries:
    df = fetch_books(q, max_total=1000)
    df['query'] = q  # чтобы потом понимать, откуда книга
    all_dfs.append(df)

combined_df = pd.concat(all_dfs, ignore_index=True)
combined_df.drop_duplicates(subset=["title", "authors"], inplace=True)

combined_df.to_csv("books_detective_expanded.csv", index=False, encoding="utf-8")
print(f"✅ Итоговая таблица содержит {len(combined_df)} уникальных книг.")


🔎 Всего найдено книг по запросу 'detective AND author:Agatha': 1000000
📥 startIndex=0, получено книг: 40
📥 startIndex=40, получено книг: 40
📥 startIndex=80, получено книг: 40
📥 startIndex=120, получено книг: 40
📥 startIndex=160, получено книг: 40
📥 startIndex=200, получено книг: 40
📥 startIndex=240, получено книг: 0
🚫 Больше результатов нет, останавливаем загрузку.
✅ Загружено всего: 240 книг

🔎 Всего найдено книг по запросу 'detective AND author:Doyle': 1000000
📥 startIndex=0, получено книг: 40
📥 startIndex=40, получено книг: 40
📥 startIndex=80, получено книг: 40
📥 startIndex=120, получено книг: 40
📥 startIndex=160, получено книг: 40
📥 startIndex=200, получено книг: 40
📥 startIndex=240, получено книг: 40
📥 startIndex=280, получено книг: 40
📥 startIndex=320, получено книг: 0
🚫 Больше результатов нет, останавливаем загрузку.
✅ Загружено всего: 320 книг

🔎 Всего найдено книг по запросу 'detective AND thriller': 1000000
📥 startIndex=0, получено книг: 40
📥 startIndex=40, получено книг: 40


In [ ]:
# print(len(df))

320


In [ ]:
# import pandas as pd
# import numpy as np
# import seaborn as sns
# import matplotlib.pyplot as plt


# # columns = [
# #     "age", "workclass", "fnlwgt", "education", "education-num",
# #     "marital-status", "occupation", "relationship", "race", "sex",
# #     "capital-gain", "capital-loss", "hours-per-week", "native-country", "salary"
# # ]

# data = pd.read_csv('books_science_fiction.csv')

# # data.replace('?', np.nan, inplace=True)

# data.head()

,title,authors,publishedDate,categories,pageCount,averageRating,ratingsCount,description
0,Science-fiction,"Everett Franklin Bleiler, Richard Bleiler",1998,Fiction,780.0,NaN,NaN,Complementing Science-Fiction: The Early Years...
1,Science/Fiction Collections,Lee Ash,2013-08-21,Fiction,190.0,NaN,NaN,Science/Fiction Collections offers different v...
2,Science Fiction,Roger Luckhurst,2005-05-06,Language Arts & Disciplines,320.0,NaN,NaN,In this new and timely cultural history of sci...
3,Science Fiction Film,Keith M. Johnston,2013-05-09,Performing Arts,193.0,NaN,NaN,Science Fiction Film develops a historical and...
4,Science Fiction Criticism,Rob Latham,2017-02-23,Literary Criticism,593.0,NaN,NaN,Including more than 30 essential works of scie...
